In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import AllTogether as t
import seaborn as sns
%matplotlib inline

demo_df= pd.read_csv('./Census_demographics_by_County.csv')
demo_state_df = pd.read_csv("./US_States.csv")
crime_df = pd.read_csv('./crime_data_w_population_and_crime_rate.csv')

In [2]:
crime_df.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,2,5,119,82,184,12,2,7629,17,3
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,3,1,2,5,4,4,0,412,48,261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,4,17,368,149,494,60,0,27083,22,31


In [3]:
demo_df.head()

,OBJECTID,STATE,STCN,CNTYNAME,AREALAND,AREAWATER,TOTALPOP,POP_DEN,NUM_MINORITY,PCT_MINORITY,...,EMP_STAT_NOT_IN_LF,PCT_EMP_STAT_NOT_IN_LF,Shape_Length,Shape_Area,FBPOP,EUROPE,ASIA,AFRICA,OCEANIA,AMERICAS
0,1,1,1001,Autauga County,1539584444,25773561,55136,92.753326,13103,23.764872,...,15561,36.603782,2.052620,0.150260,772.0,141.0,150.0,0.0,119.0,362.0
1,2,1,1005,Barbour County,2291820953,50864677,27119,30.647195,14614,53.888418,...,11086,50.736842,2.566611,0.223256,769.0,14.0,169.0,0.0,0.0,586.0
2,3,1,1023,Choctaw County,2365954803,19059247,13546,14.828676,5995,44.256607,...,5594,50.785293,2.684498,0.227716,30.0,12.0,16.0,0.0,0.0,2.0
3,4,1,1033,Colbert County,1534878355,80029923,54491,91.949330,11392,20.906205,...,20519,46.439888,2.174954,0.158871,997.0,51.0,197.0,0.0,0.0,749.0
4,5,1,1047,Dallas County,2534807336,39134779,42743,43.673480,30492,71.337997,...,15660,47.866487,3.008813,0.246517,252.0,89.0,24.0,14.0,0.0,125.0


In [4]:
demo_state_df.head()

,OBJECTID_1,OBJECTID,STATE_NAME,STATE_FIPS,STATE_ABBR,SHAPE_Length,SHAPE_Area
0,1,NaN,Alabama,1,AL,20.554796,12.881818
1,2,NaN,Alaska,2,AK,318.661159,167.784052
2,3,NaN,Arizona,4,AZ,23.916224,28.921383
3,4,NaN,Arkansas,5,AR,22.058710,13.585572
4,5,NaN,California,6,CA,55.520183,41.639766


In [5]:
#Join the two datasets on the county name.
# first clean up the county name column from crime_df

for name in range(len(crime_df['county_name'])):
    crime_df['county_name'][name] = crime_df['county_name'][name].split(",")
    crime_df['county_name'][name][1] = crime_df['county_name'][name][1].strip()
crime_df.head()

for name in range(len(demo_df['STATE'])):
    for stateCode in range(len(demo_state_df['STATE_FIPS'])):
        if(demo_df['STATE'][name] == demo_state_df['STATE_FIPS'][stateCode]):
            demo_df['STATE'][name] = demo_state_df['STATE_ABBR'][stateCode]
demo_df.head()

<ipython-input-5-6c1d0503a7e8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df['county_name'][name] = crime_df['county_name'][name].split(",")
<ipython-input-5-6c1d0503a7e8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df['STATE'][name] = demo_state_df['STATE_ABBR'][stateCode]
/home/ada/pythonJupyter/jupyterenvironment/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,OBJECTID,STATE,STCN,CNTYNAME,AREALAND,AREAWATER,TOTALPOP,POP_DEN,NUM_MINORITY,PCT_MINORITY,...,EMP_STAT_NOT_IN_LF,PCT_EMP_STAT_NOT_IN_LF,Shape_Length,Shape_Area,FBPOP,EUROPE,ASIA,AFRICA,OCEANIA,AMERICAS
0,1,AL,1001,Autauga County,1539584444,25773561,55136,92.753326,13103,23.764872,...,15561,36.603782,2.052620,0.150260,772.0,141.0,150.0,0.0,119.0,362.0
1,2,AL,1005,Barbour County,2291820953,50864677,27119,30.647195,14614,53.888418,...,11086,50.736842,2.566611,0.223256,769.0,14.0,169.0,0.0,0.0,586.0
2,3,AL,1023,Choctaw County,2365954803,19059247,13546,14.828676,5995,44.256607,...,5594,50.785293,2.684498,0.227716,30.0,12.0,16.0,0.0,0.0,2.0
3,4,AL,1033,Colbert County,1534878355,80029923,54491,91.949330,11392,20.906205,...,20519,46.439888,2.174954,0.158871,997.0,51.0,197.0,0.0,0.0,749.0
4,5,AL,1047,Dallas County,2534807336,39134779,42743,43.673480,30492,71.337997,...,15660,47.866487,3.008813,0.246517,252.0,89.0,24.0,14.0,0.0,125.0


In [6]:
crime_df.insert(0,"JoinColumn", [np.nan] * crime_df.shape[0], True )
demo_df.insert(0,"JoinColumn", [np.nan] * demo_df.shape[0], True )

In [7]:
#use a column to match rows and count the number of matches
numEqual = 0
for x in range(len(crime_df['county_name'])):
    for y in range(len(demo_df['CNTYNAME'])):
        if crime_df['county_name'][x][0] == demo_df['CNTYNAME'][y]:
            if(crime_df['county_name'][x][1] == demo_df['STATE'][y]):
                numEqual += 1
                crime_df['JoinColumn'][x] = numEqual
                demo_df['JoinColumn'][y] = numEqual

#Show the number of matches                
print(numEqual)
print(crime_df.shape)
print(demo_df.shape)
#Drop Columns that did not have a match
crime_df_dropped = crime_df.dropna(subset=['JoinColumn'], how = 'any')
demo_df_dropped = demo_df.dropna(subset=['JoinColumn'], how = 'any')
#Show the number of matches in each data frame
#Note that they are the same size, there were not any duplicate matches
print(crime_df_dropped.shape)
print(demo_df_dropped.shape)

<ipython-input-7-66bdbc8bd6e8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df['JoinColumn'][x] = numEqual
<ipython-input-7-66bdbc8bd6e8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df['JoinColumn'][y] = numEqual


3135
(3136, 25)
(3220, 267)
(3135, 25)
(3135, 267)


In [15]:
dropList = ['JoinColumn','county_name', 'EDITION', 'PART', 'IDNO', 'INDEX', 'FIPS_ST', 'FIPS_CTY','OBJECTID', 'STATE', 'STCN', 'CNTYNAME']
joinedDF = crime_df_dropped.merge(demo_df_dropped, on='JoinColumn')
columbia = joinedDF[joinedDF['STATE'] =='MD'].copy()
columbia = columbia[columbia['CNTYNAME'] == "Howard County"]

odenton = joinedDF[joinedDF['STATE'] =='MD'].copy()
odenton = odenton[odenton['CNTYNAME'] == "Anne Arundel County"]

glen_burnie = joinedDF[joinedDF['STATE'] =='MD'].copy()
glen_burnie = glen_burnie[glen_burnie['CNTYNAME'] == "Anne Arundel County"]

hanover = joinedDF[joinedDF['STATE'] =='MD'].copy()
hanover = hanover[hanover['CNTYNAME'] == "Anne Arundel County"]


columbia = columbia.drop(dropList, axis =1)
odenton = odenton.drop(dropList, axis =1)
glen_burnie = glen_burnie.drop(dropList, axis =1)
hanover = hanover.drop(dropList, axis =1)
joinedDF = joinedDF.drop(dropList, axis =1)

def imputeSubset(df, portion):
    for column in df.columns:
        if np.issubdtype(df.dtypes[column], np.number):
                df[column] = df[column] * portion

#Based on 2019 
howardCountyPop = 325690
columbiaPop = 103991
anneArundelPop = 579234
odentonPop = 41846
glen_burniePop = 69872
hanoverPop = 12952

#Impute values for the cities from their county
imputeSubset(columbia, columbiaPop / howardCountyPop)
imputeSubset(odenton, odentonPop / anneArundelPop)
imputeSubset(glen_burnie, glen_burniePop / anneArundelPop)
imputeSubset(hanover, hanoverPop / anneArundelPop)

In [9]:
joinedDF = joinedDF.drop(dropList, axis =1)

Modeling Crime

In [10]:
X = joinedDF.drop(['RAPE'], axis = 1)
y = joinedDF['RAPE']
#Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)
# train
lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit
# predict
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)
#score
test_score = r2_score(y_test, y_test_preds)
train_score = r2_score(y_train, y_train_preds)

print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 0.9737065267173043.  The rsquared on the test data was 0.753872589148916.


In [11]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.head(20)



,est_int,coefs,abs_coefs
177,PCT_HLI_API_LI,-4.305293e+11,4.305293e+11
269,PCT_EMP_STAT_NOT_IN_LF,4.028948e+11,4.028948e+11
259,PCT_EMP_STAT_IN_LF,4.028948e+11,4.028948e+11
179,PCT_HLI_API_NLI,-3.184206e+11,3.184206e+11
175,PCT_HLI_API,3.142630e+11,3.142630e+11
183,PCT_HLI_OTHER_LI,-2.651058e+11,2.651058e+11
171,PCT_HLI_IE_LI,-1.841289e+11,1.841289e+11
185,PCT_HLI_OTHER_NLI,-1.529972e+11,1.529972e+11
181,PCT_HLI_OTHER,1.488395e+11,1.488395e+11
165,PCT_HLI_SPANISH_LI,-1.317063e+11,1.317063e+11


In [12]:
columbia = columbia.drop(['RAPE'], axis = 1)
odenton =odenton.drop(['RAPE'], axis = 1)
glen_burnie = glen_burnie.drop(['RAPE'], axis = 1)
hanover = hanover.drop(['RAPE'], axis = 1)

columbiaRapeVal = lm_model.predict(columbia)
odentonRapeVal = lm_model.predict(odenton)
glen_burnieRapeVal = lm_model.predict(glen_burnie)
hanoverRapeVal = lm_model.predict(hanover)


In [14]:
print(columbiaRapeVal / columbiaPop)
print(odentonRapeVal / odentonPop)
print(glen_burnieRapeVal / glen_burniePop)
print(hanoverRapeVal / hanoverPop )

[-2.21965263e+08]
[-7.51798451e+08]
[-4.26766951e+08]
[-2.55954833e+09]
